# Run a trained policy

This notebook will provide examples on how to run a trained policy and visualize the rollout.

In [1]:
import argparse
import json
import h5py
import imageio
import numpy as np
import os
from copy import deepcopy

import torch

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.tensor_utils as TensorUtils
import robomimic.utils.obs_utils as ObsUtils
from robomimic.envs.env_base import EnvBase
from robomimic.algo import RolloutPolicy

import urllib.request


### Download policy checkpoint
First, let's try downloading a pretrained model from our model zoo.

In [2]:
# dg-noforce
dgnf = "C:\workspace\droid_policy_learning\logs\droid\im\diffusion_policy\10-05-None\bz_16_noise_samples_8_sample_weights_1_dataset_names_deligrasp_cams_workspace_wrist_goal_mode_None_truncated_geom_factor_0.3_ldkeys_proprio-lang_visenc_VisualCore_fuser_None\20241005233539\models\model_epoch_40.pth"

# dg
dg = "C:/workspace/droid_policy_learning/logs/droid/im/diffusion_policy/10-05-None/bz_16_noise_samples_8_sample_weights_1_dataset_names_deligrasp_cams_workspace_wrist_goal_mode_None_truncated_geom_factor_0.3_ldkeys_proprio-lang_visenc_VisualCore_fuser_None/20241005233539/models/model_epoch_40.pth"
# dg = "C:/workspace/droid_policy_learning/logs/droid/im/diffusion_policy/10-05-None/bz_16_noise_samples_8_sample_weights_1_dataset_names_deligrasp_cams_workspace_wrist_goal_mode_None_truncated_geom_factor_0.3_ldkeys_proprio-lang_visenc_VisualCore_fuser_None/20241005230511/models/model_epoch_25.pth"

### Loading trained policy
We have a convenient function called `policy_from_checkpoint` that takes care of building the correct model from the checkpoint and load the trained weights. Of course you could also load the checkpoint manually.

In [3]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)

# restore policy
policy, ckpt_dict = FileUtils.policy_from_checkpoint(ckpt_path=dg, device=device, verbose=True)

============= Loaded Config =============
{
    "algo_name": "diffusion_policy",
    "experiment": {
        "name": "bz_16_noise_samples_8_sample_weights_1_dataset_names_deligrasp_cams_workspace_wrist_goal_mode_None_truncated_geom_factor_0.3_ldkeys_proprio-lang_visenc_VisualCore_fuser_None",
        "validate": false,
        "logging": {
            "terminal_output_to_txt": true,
            "log_tb": true,
            "log_wandb": true,
            "wandb_proj_name": "jaf"
        },
        "mse": {
            "enabled": true,
            "every_n_epochs": 10,
            "on_save_ckpt": true,
            "num_samples": 6,
            "visualize": true
        },
        "save": {
            "enabled": true,
            "every_n_seconds": null,
            "every_n_epochs": 20,
            "epochs": [],
            "on_best_validation": false,
            "on_best_rollout_return": false,
            "on_best_rollout_success_rate": true
        },
        "epoch_every_n_steps": 1

c:\Users\willi\.conda\envs\octo_85b83fc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\willi\.conda\envs\octo_85b83fc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


number of parameters: 7.990606e+07
============= Loaded Policy =============
DiffusionPolicyUNet (
  ModuleDict(
    (policy): ModuleDict(
      (obs_encoder): DataParallel(
        (module): ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=robot_state/cartesian_position
                    shape=[6]
                    modality=low_dim
                    randomizer=ModuleList(
                      (0): None
                    )
                    net=None
                    sharing_from=None
                )
                Key(
                    name=robot_state/gripper_position
                    shape=[1]
                    modality=low_dim
                    randomizer=ModuleList(
                      (0): None
                    )
                    net=None
                    sharing_from=None
                )
                Key(
                    name=camera/image/varied_camera_1_left

### Creating rollout envionment
The policy checkpoint also contains sufficient information to recreate the environment that it's trained with. Again, you may manually create the environment.

In [4]:
ob = {
    "image": np.zeros((3, 64, 64)),
    "proprio": np.zeros((5,)),
    "lang": np.zeros((10,)),
}

In [8]:
policy.goal_mode = None
policy.eval_mode = True

In [9]:
policy(ob)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\workspace\\droid_policy_learning\\examples\\notebooks\\eval_params\\lang_command.txt'

In [16]:
policy

DiffusionPolicyUNet (
  ModuleDict(
    (policy): ModuleDict(
      (obs_encoder): DataParallel(
        (module): ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=robot_state/cartesian_position
                    shape=[6]
                    modality=low_dim
                    randomizer=ModuleList(
                      (0): None
                    )
                    net=None
                    sharing_from=None
                )
                Key(
                    name=robot_state/gripper_position
                    shape=[1]
                    modality=low_dim
                    randomizer=ModuleList(
                      (0): None
                    )
                    net=None
                    sharing_from=None
                )
                Key(
                    name=camera/image/varied_camera_1_left_image
                    shape=[3, 128, 128]
                    modality=r

### Define the rollout loop
Now let's define the main rollout loop. The loop runs the policy to a target `horizon` and optionally writes the rollout to a video.

In [ ]:
def rollout(policy, env, horizon, render=False, video_writer=None, video_skip=5, camera_names=None):
    """
    Helper function to carry out rollouts. Supports on-screen rendering, off-screen rendering to a video, 
    and returns the rollout trajectory.
    Args:
        policy (instance of RolloutPolicy): policy loaded from a checkpoint
        env (instance of EnvBase): env loaded from a checkpoint or demonstration metadata
        horizon (int): maximum horizon for the rollout
        render (bool): whether to render rollout on-screen
        video_writer (imageio writer): if provided, use to write rollout to video
        video_skip (int): how often to write video frames
        camera_names (list): determines which camera(s) are used for rendering. Pass more than
            one to output a video with multiple camera views concatenated horizontally.
    Returns:
        stats (dict): some statistics for the rollout - such as return, horizon, and task success
    """
    assert isinstance(env, EnvBase)
    assert isinstance(policy, RolloutPolicy)
    assert not (render and (video_writer is not None))

    policy.start_episode()
    obs = env.reset()
    state_dict = env.get_state()

    # hack that is necessary for robosuite tasks for deterministic action playback
    obs = env.reset_to(state_dict)

    results = {}
    video_count = 0  # video frame counter
    total_reward = 0.
    try:
        for step_i in range(horizon):

            # get action from policy
            act = policy(ob=obs)

            # play action
            next_obs, r, done, _ = env.step(act)

            # compute reward
            total_reward += r
            success = env.is_success()["task"]

            # visualization
            if render:
                env.render(mode="human", camera_name=camera_names[0])
            if video_writer is not None:
                if video_count % video_skip == 0:
                    video_img = []
                    for cam_name in camera_names:
                        video_img.append(env.render(mode="rgb_array", height=512, width=512, camera_name=cam_name))
                    video_img = np.concatenate(video_img, axis=1) # concatenate horizontally
                    video_writer.append_data(video_img)
                video_count += 1

            # break if done or if success
            if done or success:
                break

            # update for next iter
            obs = deepcopy(next_obs)
            state_dict = env.get_state()

    except env.rollout_exceptions as e:
        print("WARNING: got rollout exception {}".format(e))

    stats = dict(Return=total_reward, Horizon=(step_i + 1), Success_Rate=float(success))

    return stats


### Run the policy
Now let's rollout the policy!

In [ ]:
rollout_horizon = 400
np.random.seed(0)
torch.manual_seed(0)
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

In [ ]:
stats = rollout(
    policy=policy, 
    env=env, 
    horizon=rollout_horizon, 
    render=False, 
    video_writer=video_writer, 
    video_skip=5, 
    camera_names=["agentview"]
)
print(stats)
video_writer.close()

### Visualize the rollout

In [ ]:
from IPython.display import Video
Video(video_path)